In [1]:
import pickle
import mediapipe as mp
import pandas as pd
import numpy as np

from sklearn import metrics, model_selection, ensemble

import time
import cv2
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
import glob
import pickle

In [2]:
with open('../models/random_forest_best/random_forest_best.pkl', 'rb') as file:
    clf = pickle.load(file)

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [4]:
int2char = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 
    10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 
    19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z', 26: 'd', 27: 's', 28: 'n'
}

In [ ]:
cap = cv2.VideoCapture(0)

clas = None
start_time = time.time()
last_prediction_time = start_time

try:
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        
        while cap.isOpened():
            
            ret, frame = cap.read()
            if not ret:
                print("No image from camera")
                break
    
    
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # frame_rgb.flags.writeable = False
            
            results = hands.process(frame_rgb)
    
            points = []
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    for i in range(21):
                        x = hand_landmarks.landmark[i].x
                        y = hand_landmarks.landmark[i].y
                        z = hand_landmarks.landmark[i].z
                        points.append([x, y, z])
            else:
                points = None
                # print('Points are None')
        
        
            if points is not None:
                if len(points) > 21:
                    points = points[:21]
                    print('Landmarks are more than 21')
                elif len(points) < 21:
                    dif = 21 - len(points)
                    for i in range(dif):
                        points.append([0, 0, 0])
                    print('Landmarks are less than 21')
        
                points_raw = np.array(points)
    
            
                min_x = np.min(points_raw[:, 0])
                max_x = np.max(points_raw[:, 0])
                min_y = np.min(points_raw[:, 1])
                max_y = np.max(points_raw[:, 1])
                for i in range(len(points_raw)):
                    points_raw[i][0] = (points_raw[i][0] - min_x) / (max_x - min_x)
                    points_raw[i][1] = (points_raw[i][1] - min_y) / (max_y - min_y)
    
            
                current_time = time.time()
                if current_time - last_prediction_time >= 2: 
                
                    flattened_points = [item for sublist in points_raw for item in sublist]
                    X = np.array(flattened_points).reshape(1, -1)
                    prediction = clf.predict(X)
                    clas = int2char[prediction[0]]
        
                    h, w, c = frame.shape
                    # frame.flags.writeable = True
                    # cv2.putText(frame, f'Prediciton: {clas}', (10, h - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                    print(clas)
                    last_prediction_time = current_time
    
            # image.flags.writeable = True
            
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)


            elaps = time.time()
            elapsed_time = elaps - start_time  
            elapsed_time_text = f'Timer: {int(elapsed_time)} second'  
        
            cv2.putText(frame, elapsed_time_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'Prediciton: {clas}', (10, h - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


            
            cv2.imshow('Sign Detection', frame)

            
     
    
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyWindow("Sign Detection")
                cv2.waitKey(1)
                cap.release()
                cv2.waitKey(1)
                cv2.destroyAllWindows()
                cv2.waitKey(1)
                break

finally:
    cv2.waitKey(1)
    cv2.destroyWindow("Sign Detection") 
    cv2.waitKey(1)
    cap.release()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)


In [29]:
cap.release()
cv2.destroyAllWindows()